In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import statsrat as sr
import decision
import glob

In [2]:
# list .txt files in the directory
path = 'decision_data'
file_set = [file for file in glob.glob(path + "**/*.txt", recursive=True)]
assert len(file_set) > 0, 'Cannot find any files in specified path.'
n_f = len(file_set)

In [3]:
# MAKE PANDAS DATAFRAME    
# loop through files
date, ident, group, trial_type, color, prob, ambig, lottery_value, ref_value, lottery_side, resp = [[], [], [], [], [], [], [], [], [], [], []]
for i in range(n_f):
    # import raw data
    raw = pd.read_table(file_set[i], error_bad_lines=False)
    # only proceed if the file has the right columns
    col_names = list(raw.columns)
    if ('CPGroup' in col_names) and ('RiskAmbigLevel' in col_names):
        raw.dropna(subset = ['RiskAmbigLevel', 'choice.RESP'], thresh = 1, inplace = True) # drop rows without recorded trial type ('RiskAmbigLevel') or response ('choice.RESP')
        n_r = raw.shape[0] # number of rows in raw data frame
        raw.index = range(n_r) # re-index 'raw'
        date += n_r*[raw['SessionDate'].iloc[0]]
        new_ident = str(raw['Subject'].iloc[0]) + '_' + str(raw['CPGroup'].iloc[0])
        ident += n_r*[new_ident]
        group += n_r*[raw['CPGroup'].iloc[0]]
        new_trial_type, new_color, new_resp = [n_r*[''], n_r*[''], n_r*['']]
        new_prob, new_ambig, new_lottery_value, new_ref_value, new_lottery_side = [n_r*[0], n_r*[0], n_r*[0], n_r*[0], n_r*[0]]
        # loop through rows (trials)
        for j in range(n_r):
            (new_trial_type[j], new_color[j], level) = raw['RiskAmbigLevel'].iloc[j].split('_')
            if new_trial_type[j] == 'risk':
                # risky trial
                new_prob[j] = int(level)/100
                new_ambig[j] = 0
            else:
                # ambiguous trials
                new_prob[j] = 0.5
                new_ambig[j] = int(level)/100
            if new_color[j] == 'red':
                # betting on red
                new_lottery_value[j] = float(raw['RedValue'].iloc[j].replace('$', ''))
            else:
                # betting on blue
                new_lottery_value[j] = float(raw['BlueValue'].iloc[j].replace('$', ''))
            new_ref_value[j] = float(raw['Reference'].iloc[j].replace('$', ''))
            new_lottery_side[j] = raw['LotterySide'].iloc[j].lower()
            if raw['choice.RESP'].iloc[j] == 1:
                new_resp[j] = 'left'
            else:
                new_resp[j] = 'right'
        trial_type += new_trial_type
        color += new_color
        prob += new_prob
        ambig += new_ambig
        lottery_value += new_lottery_value
        ref_value += new_ref_value
        lottery_side += new_lottery_side
        resp += new_resp   

# create new dataframe
df = pd.DataFrame({'ident': ident,
                   'date': date,
                   'group': group,
                   'trial_type': trial_type,
                   'color': color,
                   'prob': prob,
                   'ambig': ambig,
                   'lottery_value': lottery_value,
                   'ref_value': ref_value,
                   'lottery_side': lottery_side,
                   'resp': resp})

# print full data frame
with pd.option_context('display.max_rows', 
                        None,
                        'display.max_columns', 
                        None):  # more options can be specified also
    print(df)

         ident        date   group trial_type color  prob  ambig  \
0     24_Child  08-12-2018   Child       risk   red  0.75   0.00   
1     24_Child  08-12-2018   Child       risk   red  0.25   0.00   
2     24_Child  08-12-2018   Child       risk  blue  0.25   0.00   
3     24_Child  08-12-2018   Child      ambig  blue  0.50   0.75   
4     24_Child  08-12-2018   Child      ambig  blue  0.50   0.25   
5     24_Child  08-12-2018   Child       risk   red  0.38   0.00   
6     24_Child  08-12-2018   Child       risk  blue  0.13   0.00   
7     24_Child  08-12-2018   Child      ambig   red  0.50   0.50   
8     24_Child  08-12-2018   Child       risk  blue  0.75   0.00   
9     24_Child  08-12-2018   Child       risk  blue  0.50   0.00   
10    24_Child  08-12-2018   Child      ambig   red  0.50   0.75   
11    24_Child  08-12-2018   Child       risk   red  0.13   0.00   
12    24_Child  08-12-2018   Child      ambig  blue  0.50   0.50   
13    24_Child  08-12-2018   Child       risk  b

In [4]:
# MAKE XARRAY DATASET
idents = np.unique(df.ident)
n = len(idents) # number of participants
n_c = 2 # number of choices
n_t = n_r#df.loc[df.ident == idents[0], :].shape[0] # number of trials per participant
# column 0 = left; column 1 = right

# loop through participants ('ident')
ds_list = []
for i in range(n):
    dfi = df.loc[df.ident == idents[i], :] # data from individual i
    v, p, a, b = [np.zeros((n_t, n_c)), np.zeros((n_t, n_c)), np.zeros((n_t, n_c)), np.zeros((n_t, n_c))] # empty arrays for data
    # loop through rows (trials)
    for j in range(n_r):
        if dfi.lottery_side.iloc[j] == 'left':
            # lottery on left
            v[j, 0] = dfi.lottery_value.iloc[j]
            p[j, 0] = dfi.prob.iloc[j]
            a[j, 0] = dfi.ambig.iloc[j]
            # reference on right
            v[j, 1] = dfi.ref_value.iloc[j]
            p[j, 1] = 1
            a[j, 1] = 0
        else:
            # reference on left
            v[j, 0] = dfi.ref_value.iloc[j]
            p[j, 0] = 1
            a[j, 0] = 0
            # lottery on right
            v[j, 1] = dfi.lottery_value.iloc[j]
            p[j, 1] = dfi.prob.iloc[j]
            a[j, 1] = dfi.ambig.iloc[j]
        if dfi.resp.iloc[j] == 'left':
            # participant chose left
            b[j, :] = [1, 0]
        else:
            # participant chose right
            b[j, :] = [0, 1]
            
    # create an xarray dataset for this individual
    ds_new = xr.Dataset(data_vars = {'v' : (['t', 'choice'], v),
                                     'p' : (['t', 'choice'], p),
                                     'a' : (['t', 'choice'], a),
                                     'b' : (['t', 'choice'], b)},
                        coords = {'t' : range(n_t),
                                  'choice' : ['left', 'right'],
                                  'ident' : [idents[i]]})
    ds_list += [ds_new]

ds = xr.combine_nested(ds_list, concat_dim = 'ident')
with pd.option_context('display.max_rows', 
                        None,
                        'display.max_columns', 
                        None):  # more options can be specified also
    print(ds.loc[{'ident': idents[0]}].to_dataframe())

               v     p     a    b     ident
choice t                                   
left   0    12.0  0.75  0.00  0.0  14_Child
       1     1.5  1.00  0.00  0.0  14_Child
       2     1.5  1.00  0.00  0.0  14_Child
       3     2.0  0.50  0.75  0.0  14_Child
       4    -1.5  1.00  0.00  0.0  14_Child
       5    -2.0  0.38  0.00  0.0  14_Child
       6    -1.5  1.00  0.00  0.0  14_Child
       7    -7.0  0.50  0.50  0.0  14_Child
       8     1.5  1.00  0.00  0.0  14_Child
       9     1.5  1.00  0.00  0.0  14_Child
       10   20.0  0.50  0.75  0.0  14_Child
       11    1.5  1.00  0.00  0.0  14_Child
       12   -1.5  1.00  0.00  0.0  14_Child
       13   -1.5  1.00  0.00  0.0  14_Child
       14   -1.5  1.00  0.00  0.0  14_Child
       15   -6.0  0.50  0.25  0.0  14_Child
       16    1.5  1.00  0.00  1.0  14_Child
       17    2.5  0.75  0.00  0.0  14_Child
       18    1.5  1.00  0.00  0.0  14_Child
       19    1.5  1.00  0.00  1.0  14_Child
       20    1.5  0.50  0.00  0.

In [10]:
# ESTIMATE INDIVIDUAL PARAMETERS
dmodel = decision.model()
model_fit = sr.fit_indv(model = dmodel, ds = ds, a = 1, b = 1)
print(model_fit)

lvl0   est_par                      prop_log_post      ident
lvl1     alpha      beta resp_scale                         
0     0.530498  0.053691   1.420469    -74.007014   14_Child
1     1.029543  0.309840   0.433012    -71.138621  14_Parent
2     0.489593  0.017099   1.091317    -79.251745   24_Child
3     0.587765  0.370827   1.132461    -77.376304   28_Child
